### <a class="anchor" id="Rank">Executive Summary</a>

The goal of this project is to build a Restaurant Recommender System, which offers restuarant recommendation to users in the city of London

The metrics of the Recommender System is the accuracy of the systems models built:
I. Rank-based Recommender System
II. Content-based Recommender System
III. Collaborative-Fitering Recommender System with Surpise - SVD

The limitations of this project are 
I. Its is computationally expensive
II. Extremely large data size
      
_

The four notebooks used in this project are

I. Web Scraping the Data

II. Merging the Dataframes

III. Exploratory Data Analysis

IV. Restaurant Recommender System & Models

### I. Web Scraping the Data

The first data scraped was a list of restaurants in London. To get the list of users who had reviewd these resturants, I had to use the links ('hrefs') from the original list of retaurants scraped.

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import re



In [4]:
restau_lon = pd.read_csv('restau_london_final.csv')
del restau_lon['Unnamed: 0']
restau_lon.head()

,Name,Url,Review,Rating,Category 1,Category 2
0,Amrutha Lounge,https://www.tripadvisor.co.uk/Restaurant_Revie...,425 reviews,5 of 5 bubbles,Indian,££ - £££\nIndian\nAsian\nHealthy\nVegetarian F...
1,Barge East,https://www.tripadvisor.co.uk/Restaurant_Revie...,184 reviews,5 of 5 bubbles,European,££ - £££\nEuropean\nBritish\nContemporary\nFus...
2,Latitude,https://www.tripadvisor.co.uk/Restaurant_Revie...,78 reviews,5 of 5 bubbles,European,££ - £££\nEuropean\nInternational\nBritish\nVe...
3,Dobar,https://www.tripadvisor.co.uk/Restaurant_Revie...,107 reviews,5 of 5 bubbles,Pizza,££ - £££\nPizza\nCafe\nBritish\nVegetarian Fri...
4,Chojo,https://www.tripadvisor.co.uk/Restaurant_Revie...,199 reviews,5 of 5 bubbles,Chinese,££ - £££\nChinese\nTaiwanese\nStreet Food\nVeg...


Below, we see the column of all links (hrefs) 

In [5]:
restau_lon['Url'].head()

0    https://www.tripadvisor.co.uk/Restaurant_Revie...
1    https://www.tripadvisor.co.uk/Restaurant_Revie...
2    https://www.tripadvisor.co.uk/Restaurant_Revie...
3    https://www.tripadvisor.co.uk/Restaurant_Revie...
4    https://www.tripadvisor.co.uk/Restaurant_Revie...
Name: Url, dtype: object

In [8]:
restau_lon.Review = [int(x.replace(',', '').replace('reviews','').replace('review','')) for x in restau_lon.Review]
restau_lon.Rating = [float(str(x).replace('of 5 bubbles', '')) for x in restau_lon.Rating]
restau_lon = restau_lon[restau_lon.Rating >= 3.0]
pages = [string.replace('Reviews-','Reviews-or{}-') for string in restau_lon['Url']]

In [11]:
pages[1:4]

['https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d14766290-Reviews-or{}-Barge_East-London_England.html',
 'https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d13474320-Reviews-or{}-Latitude-London_England.html',
 'https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d15211822-Reviews-or{}-Dobar-London_England.html']

## Data Scraping 1/90
Scraping & Saving Restaurants User Rating List

-------



Now, we parse through the list of links to users per restaurants. Scraping the other Dataframes is the same appoach, all that changes is the numbers specifying range of links to parse in 'pages'.

In [10]:
user_id = []
restau_id = []
rating = []
count = 0

for flame in tqdm_notebook(pages[1160:1300]):
    r = requests.get(flame)
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        maxx = int(re.findall('\d+', (str(soup.find('div',attrs={'class':'pagination-details'}))))[-1])
    except:
        maxx = 10
    for q in tqdm_notebook(range(0,maxx,10)):
        url = flame.format(q)
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        for restau in soup.find_all('div', attrs={'class':'rev_wrap ui_columns is-multiline'}):

            try:
                user_id.append(restau.find('div', attrs={'class':'info_text'}).text)
            except:
                user_id.append(np.nan)
            try:
                rating.append(restau.find('span', attrs={'class':'ui_bubble_rating'}))
            except:
                rating.append(np.nan)
                
            for restau in soup.find_all('div', attrs={'class':'restaurantName ui_column is-10-tablet is-11-mobile'}):
                try:
                    restau_id.append(restau.find('h1', attrs={'class':'ui_header h1'}).text)
                except:
                    restau_id.append(np.nan)
    count += 1
    
    if count == 10:
        restau_ = pd.DataFrame({'Name': user_id, 'Restaurant': restau_id, 'Rating': rating})
        restau_.to_csv('Nast_12.csv')
        count = 0
restau_ = pd.DataFrame({'Name': user_id, 'Restaurant': restau_id, 'Rating': rating})
restau_.to_csv('Nast_12.csv')

In [24]:
restau_

,Name,Restaurant,Rating
0,"crb3590Bexleyheath, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_50""></span>"
1,Knowswhathelikes03London,Pope Street,"<span class=""ui_bubble_rating bubble_40""></span>"
2,"Tracey MLondon, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_50""></span>"
3,Chownie2012,Pope Street,"<span class=""ui_bubble_rating bubble_50""></span>"
4,"markrob105Bexley, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_40""></span>"
5,"KarenandnicklWorthing, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_40""></span>"
6,allydc,Pope Street,"<span class=""ui_bubble_rating bubble_50""></span>"
7,"Monica MSidcup, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_40""></span>"
8,"edwardb748London, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_30""></span>"
9,"WeloveramsgateRamsgate, United Kingdom",Pope Street,"<span class=""ui_bubble_rating bubble_20""></span>"
